In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib
import gymnasium as gym
import ray

from rl_trading.simulation.env import StockExchangeEnv0

Using matplotlib backend: <object object at 0x7f7ff43e2960>


In [5]:
from ray.rllib.algorithms.r2d2 import R2D2Config

algo = (
    R2D2Config()
    .rollouts(num_rollout_workers=8)
    .environment('CartPole-v1')
    .training(replay_buffer_config={'capacity': 100_000}, model={'use_lstm': True})
    .build()
)

render_env = gym.make('CartPole-v1', render_mode='human')

for i in range(10):
    result = algo.train()
    print(f'Mean return: {result["episode_reward_mean"]}')
    if i % 5 == 0:
        state, done = render_env.reset()[0], False
        while not done:
            render_env.render()
            action = algo.compute_single_action(state)
            state, reward, terminated, truncated, _ = render_env.step(action)
            done = terminated or truncated

(RolloutWorker pid=816406)     or model.view_requirements.get("state_in_0") is not None [repeated 7x across cluster]
(RolloutWorker pid=816406) AssertionError: R2D2 requires its model to be a recurrent one! Try using `model.use_lstm` or `model.use_attention` in your config to auto-wrap your model with an LSTM- or attention net. [repeated 7x across cluster]
(RolloutWorker pid=817840) /home/fassty/anaconda3/envs/diploma_thesis/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24) [repeated 8x across cluster]
(RolloutWorker pid=817840)   if not isinstance(terminated, (bool, np.bool8)): [repeated 8x across cluster]
(RolloutWorker pid=816406) 2023-05-13 22:02:06,075	ERROR worker.py:844 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=816406, ip=192.168.0.222, repr=<ray.rllib.evaluation.rollout

(autoscaler +3m42s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +3m42s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


KeyboardInterrupt: 

In [13]:
exchange_data = pd.read_hdf('../data/binance_BTC_USDT.h5')
exchange_data.sort_index(inplace=True)
exchange_data = exchange_data[~exchange_data.index.duplicated(keep='first')]
exchange_data = exchange_data.reindex(np.arange(exchange_data.index[0], exchange_data.index[-1] + 1, 60))
exchange_data['price'] = exchange_data['price'].ffill()
exchange_data['amount'] = exchange_data['amount'].fillna(value=0)

display(exchange_data.head(5))

exchange_data.index = pd.to_datetime(exchange_data.index * 1e9)
price_data_1m = exchange_data['price'].to_numpy()
volume_data_1m = exchange_data['amount'].to_numpy()

agg_mapping = {'price': 'last', 'amount': 'sum'}
hourly_data = exchange_data.groupby(pd.Grouper(freq='1h')).agg(agg_mapping)
daily_data = exchange_data.groupby(pd.Grouper(freq='1d')).agg(agg_mapping)
weekly_data = exchange_data.groupby(pd.Grouper(freq='1w')).agg(agg_mapping)
biweekly_data = exchange_data.groupby(pd.Grouper(freq='2w')).agg(agg_mapping)
monthly_data = exchange_data.groupby(pd.Grouper(freq='1M')).agg(agg_mapping)

display(hourly_data.head(5))
display(daily_data.head(5))
display(weekly_data.head(5))
display(biweekly_data.head(5))
display(monthly_data.head(5))

,amount,price
timestamp,,
1502942460,1.775183,4261.48
1502942520,0.000000,4261.48
1502942580,0.261074,4280.56
1502942640,0.012008,4261.48
1502942700,0.140796,4261.48


,price,amount
timestamp,,
2017-08-17 04:00:00,4311.749242,46.620743
2017-08-17 05:00:00,4315.320000,23.795182
2017-08-17 06:00:00,4324.350000,7.229691
2017-08-17 07:00:00,4334.000000,2.482438
2017-08-17 08:00:00,4360.690000,2.933618


,price,amount
timestamp,,
2017-08-17,4285.08,795.150377
2017-08-18,4115.40,1199.699723
2017-08-19,4139.98,381.498304
2017-08-20,4069.13,466.704213
2017-08-21,4016.00,691.925435


,price,amount
timestamp,,
2017-08-20,4069.130000,2843.052617
2017-08-27,4346.972375,4599.745731
2017-09-03,4505.581286,4753.873083
2017-09-10,4150.230425,6381.632307
2017-09-17,3699.015885,8107.831437


,price,amount
timestamp,,
2017-08-20,4069.130000,2843.052617
2017-09-03,4505.581286,9353.618814
2017-09-17,3699.015885,14489.463744
2017-10-01,4378.480000,11690.657902
2017-10-15,5699.844347,12620.142176


,price,amount
timestamp,,
2017-08-31,4724.885015,10015.640272
2017-09-30,4347.630427,27634.043533
2017-10-31,6476.467415,41625.312848
2017-11-30,9846.140933,108483.883600
2017-12-31,13735.118870,408477.194296


In [14]:
ohlcv_1h = exchange_data['price'].groupby(pd.Grouper(freq='1h')).agg(open='first', high='max', low='min', close='last')
ohlcv_1h['volume'] = exchange_data['amount'].groupby(pd.Grouper(freq='1h')).agg('sum')
ohlcv_1h = ohlcv_1h.loc[pd.Timestamp('2017-08-18 00:00:00'):]

In [15]:
ohlcv_1h['tp'] = (ohlcv_1h['high'] + ohlcv_1h['low'] + ohlcv_1h['close']) / 3
ohlcv_1h

,open,high,low,close,volume,tp
timestamp,,,,,,
2017-08-18 00:00:00,4285.080000,4319.986355,4249.317527,4273.052256,82.378434,4280.785380
2017-08-18 01:00:00,4286.530000,4286.530000,4162.771089,4244.618032,75.741923,4231.306374
2017-08-18 02:00:00,4244.618032,4288.781267,4234.441705,4267.590000,53.841434,4263.604324
2017-08-18 03:00:00,4267.590000,4308.700000,4244.770000,4279.236936,63.688138,4277.568979
2017-08-18 04:00:00,4279.702720,4292.390000,4237.032409,4287.920000,51.340629,4272.447470
...,...,...,...,...,...,...
2022-12-08 02:00:00,16830.380506,16872.627489,16811.416299,16843.812041,8047.727090,16842.618610
2022-12-08 03:00:00,16843.887596,16862.316315,16828.768225,16832.783395,7780.884650,16841.289312
2022-12-08 04:00:00,16834.000203,16846.291704,16812.830201,16825.994048,8179.241020,16828.371985


In [16]:
def vwap(df):
    df['vwap'] = (df['tp'] * df['volume']).cumsum() / df['volume'].cumsum()
    return df

ohlcv_1h.groupby(ohlcv_1h.index.date).apply(vwap).droplevel(0)

,open,high,low,close,volume,tp,vwap
timestamp,,,,,,,
2017-08-18 00:00:00,4285.080000,4319.986355,4249.317527,4273.052256,82.378434,4280.785380,4280.785380
2017-08-18 01:00:00,4286.530000,4286.530000,4162.771089,4244.618032,75.741923,4231.306374,4257.084225
2017-08-18 02:00:00,4244.618032,4288.781267,4234.441705,4267.590000,53.841434,4263.604324,4258.740427
2017-08-18 03:00:00,4267.590000,4308.700000,4244.770000,4279.236936,63.688138,4277.568979,4263.090710
2017-08-18 04:00:00,4279.702720,4292.390000,4237.032409,4287.920000,51.340629,4272.447470,4264.559811
...,...,...,...,...,...,...,...
2022-12-08 02:00:00,16830.380506,16872.627489,16811.416299,16843.812041,8047.727090,16842.618610,16833.025808
2022-12-08 03:00:00,16843.887596,16862.316315,16828.768225,16832.783395,7780.884650,16841.289312,16834.805009
2022-12-08 04:00:00,16834.000203,16846.291704,16812.830201,16825.994048,8179.241020,16828.371985,16833.617732


In [40]:
pd.Timestamp('2017-08-17 04:03:00').floor(freq='1d')

Timestamp('2017-08-17 00:00:00')

In [11]:
exchange_data = exchange_data.loc[pd.Timestamp('2017-08-18 00:00:00'):]

In [35]:
def vwap(df):
    df['vwap'] = (df['price'] * df['volume']).cumsum() / df['volume'].cumsum()
    return df

exchange_data.groupby(exchange_data.index.date).apply(vwap).droplevel(0).ffill().bfill()

2.67 s ± 2.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
from rl_trading.simulation.env import SimulationConfig

SimulationConfig()

SimulationConfig(granularity={}, max_steps=1440, initial_cash=10000)

In [3]:
from rl_trading.data.provider import MarketDataProvider
from rl_trading.data.indicators import *

env = StockExchangeEnv0(
    state_config={
        'market_state': ['price', 'vwap'],
        'technical_indicators': [
            (EMA, dict(timeperiod=5), '1min'),
            (EMA, dict(timeperiod=13), '1min'),
            (RSI, dict(timeperiod=7), '1min'),
            (BBANDS, dict(timeperiod=10), '1min'),
            (EMA, dict(timeperiod=20), '1h'),
            (EMA, dict(timeperiod=50), '1h'),
            (RSI, dict(timeperiod=14), '1h'),
            (BBANDS, dict(timeperiod=20), '1h'),
            (MACD_DIFF, dict(fastperiod=12, slowperiod=26, signalperiod=9), '1h'),
            (EMA, dict(timeperiod=50), '1d'),
            (EMA, dict(timeperiod=200), '1d'),
            (RSI, dict(timeperiod=14), '1d'),
            (BBANDS, dict(timeperiod=20), '1d'),
            (MACD_DIFF, dict(fastperiod=12, slowperiod=26, signalperiod=9), '1d'),
        ],
    })
env.market_data.keys()
env.reset()

(array([ 1.65069189e+04,  1.65337229e+04,  1.65026921e+04,  1.64990000e+04,
         7.08373999e+01,  1.65120371e+04,  1.64793043e+04,  1.65759318e+04,
         1.65456385e+04,  4.81486633e+01,  1.66394564e+04,  1.65062512e+04,
        -5.05073731e+00,  1.86984973e+04,  2.19959123e+04,  4.21926302e+01,
         1.97912144e+04,  1.44120051e+04,  2.31243044e+02,  1.00000000e+04,
         0.00000000e+00]),
 {})

3000

In [5]:
import torch
import ray
from ray.rllib.algorithms.ppo import PPOConfig
from rl_trading.simulation.env import StockExchangeEnv0
from ray.tune.registry import register_env

ray.shutdown()
ray.init()

register_env('StockExchangeEnv-v0', lambda config: StockExchangeEnv0(**config))

ppo = (
    PPOConfig()
    .rollouts(num_rollout_workers=4, num_envs_per_worker=4)
    .training(use_gae=True, model={'use_lstm': True, 'lstm_use_prev_action': True, 'lstm_use_prev_reward': True})
    .resources(num_gpus=1)
    .environment(env='StockExchangeEnv-v0', env_config={'state_config':{
        'market_state': ['price', 'vwap'],
        'technical_indicators': [
            (EMA, dict(timeperiod=5), '1min'),
            (EMA, dict(timeperiod=13), '1min'),
            (RSI, dict(timeperiod=7), '1min'),
            (BBANDS, dict(timeperiod=10), '1min'),
            (EMA, dict(timeperiod=20), '1h'),
            (EMA, dict(timeperiod=50), '1h'),
            (RSI, dict(timeperiod=14), '1h'),
            (BBANDS, dict(timeperiod=20), '1h'),
            (MACD_DIFF, dict(fastperiod=12, slowperiod=26, signalperiod=9), '1h'),
            (EMA, dict(timeperiod=50), '1d'),
            (EMA, dict(timeperiod=200), '1d'),
            (RSI, dict(timeperiod=14), '1d'),
            (BBANDS, dict(timeperiod=20), '1d'),
            (MACD_DIFF, dict(fastperiod=12, slowperiod=26, signalperiod=9), '1d'),
        ],
    }})
    .reporting(min_sample_timesteps_per_iteration=16 * 1440)
    .build()
)

render_env = StockExchangeEnv0()

for i in range(250):
    result = ppo.train()
    print(f'Step: {result["training_iteration"]:<5}Mean return: {result["episode_reward_mean"]}\t +- {result["episode_reward_max"] - result["episode_reward_min"]}')

2023-05-11 19:05:37,883	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(RolloutWorker pid=1043977) 2023-05-11 19:05:48,486	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.


Step: 1    Mean return: 11.054210175418575	 +- 682.8472323183978
Step: 2    Mean return: 22.51906906119217	 +- 701.8839957572454
Step: 3    Mean return: 41.6492745353601	 +- 1421.322676290094
Step: 4    Mean return: 78.35231343930818	 +- 1421.322676290094
Step: 5    Mean return: 94.95856714453976	 +- 1476.5776312627913
Step: 6    Mean return: 65.45305453670181	 +- 1671.729725747693
Step: 7    Mean return: 89.62004549320326	 +- 1671.729725747693
Step: 8    Mean return: 79.98371282023513	 +- 1671.729725747693
Step: 9    Mean return: 57.37391732418186	 +- 1624.297522403509
Step: 10   Mean return: 51.99980404636202	 +- 1624.297522403509
Step: 11   Mean return: 42.257349389383656	 +- 1232.9795344705617
Step: 12   Mean return: 58.63088455451591	 +- 1298.0162584396148
Step: 13   Mean return: 48.12888751001268	 +- 1192.3565403888624
Step: 14   Mean return: 43.250097994843216	 +- 1808.8996616474542
Step: 15   Mean return: 48.581189434795235	 +- 2361.2751753598823
Step: 16   Mean return: 49.0057

In [48]:
from tqdm import tqdm

env = StockExchangeEnv0(state_config={
        'market_state': ['price', 'vwap'],
        'technical_indicators': [
            (EMA, dict(timeperiod=5), '1min'),
            (EMA, dict(timeperiod=13), '1min'),
            (RSI, dict(timeperiod=7), '1min'),
            (BBANDS, dict(timeperiod=10), '1min'),
            (EMA, dict(timeperiod=20), '1h'),
            (EMA, dict(timeperiod=50), '1h'),
            (RSI, dict(timeperiod=14), '1h'),
            (BBANDS, dict(timeperiod=20), '1h'),
            (MACD_DIFF, dict(fastperiod=12, slowperiod=26, signalperiod=9), '1h'),
            (EMA, dict(timeperiod=50), '1d'),
            (EMA, dict(timeperiod=200), '1d'),
            (RSI, dict(timeperiod=14), '1d'),
            (BBANDS, dict(timeperiod=20), '1d'),
            (MACD_DIFF, dict(fastperiod=12, slowperiod=26, signalperiod=9), '1d'),
        ],
    })

def eval_episode():
    for _ in tqdm(range(10)):
        obs, done = env.reset()[0], False
        pa, pr = 0, 0
        total_reward = 0
        state = [np.zeros([256], np.float32) for _ in range(2)]
        while not done:
            # env.render()
            pa, state, _ = ppo.compute_single_action(observation=obs, state=state, prev_action=pa, prev_reward=pr)
            obs, pr, done, _, _ = env.step(pa)
            total_reward += pr

%lprun -f eval_episode eval_episode()

100%|██████████| 10/10 [00:51<00:00,  5.15s/it]


In [47]:
env = StockExchangeEnv0(state_config={
        'market_state': ['price', 'vwap'],
        'technical_indicators': [
            (EMA, dict(timeperiod=5), '1min'),
            (EMA, dict(timeperiod=13), '1min'),
            (RSI, dict(timeperiod=7), '1min'),
            (BBANDS, dict(timeperiod=10), '1min'),
            (EMA, dict(timeperiod=20), '1h'),
            (EMA, dict(timeperiod=50), '1h'),
            (RSI, dict(timeperiod=14), '1h'),
            (BBANDS, dict(timeperiod=20), '1h'),
            (MACD_DIFF, dict(fastperiod=12, slowperiod=26, signalperiod=9), '1h'),
            (EMA, dict(timeperiod=50), '1d'),
            (EMA, dict(timeperiod=200), '1d'),
            (RSI, dict(timeperiod=14), '1d'),
            (BBANDS, dict(timeperiod=20), '1d'),
            (MACD_DIFF, dict(fastperiod=12, slowperiod=26, signalperiod=9), '1d'),
        ],
    })

state = env.reset()[0]

def step_bench():
    for _ in range(100):
        env._get_observation()
%lprun -f env._get_observation step_bench()

In [2]:
from ray import tune, air
from ray.rllib.algorithms.dqn import DQNConfig
ray.shutdown()
ray.init()

from rl_trading.simulation.env import StockExchangeEnv0

config = (
    DQNConfig()
    .rollouts(num_rollout_workers=1, num_envs_per_worker=4)
    .resources(num_gpus=0.2)
    .environment(env='StockExchangeEnv-v0')
    .training(hiddens=[512], num_steps_sampled_before_learning_starts=20_000, target_network_update_freq=4_000)
)

tuner = tune.Tuner(
    "DQN",
    run_config=air.RunConfig(
        name='DQN_fixed_seed',
        local_dir='../exp_results/DQN',
        stop={"training_iteration": 250},
    ),
    tune_config=tune.TuneConfig(
        num_samples=5
    ),
    param_space=config,
)

tuner.fit()

2023-05-08 17:48:06,173	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
/home/fassty/anaconda3/envs/diploma_thesis/lib/python3.10/site-packages/ray/tune/experiment/experiment.py:170: UserWarning: The `local_dir` argument of `Experiment is deprecated. Use `storage_path` or set the `TUNE_RESULT_DIR` environment variable instead.
  warnings.warn(


(DQN pid=679423) 2023-05-08 17:48:11,273	WARNING algorithm_config.py:635 -- Cannot create DQNConfig from given `config_dict`! Property __stdout_file__ not supported.
(DQN pid=679423) 2023-05-08 17:48:11,714	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=679578) 2023-05-08 17:48:16,100	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(DQN pid=679423) 2023-05-08 17:48:16,402	WARNING multi_agent_prioritized_replay_buffer.py:215 -- Adding batches with column `weights` to this buffer while providing weights as a call argument to the add method results in the column being overwritten.
(DQN pid=679770) 2023-05-08 17:48:19,010	WARNING algorithm_config.py:635 -- Cannot create DQNConfig from given `config_dict`! Property __stdout_file__ not supported.
(DQN pid=679770) 2023-05-08 

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_total,training_iteration,trial_id
DQN_StockExchangeEnv-v0_b9c9d_00000,1000,{},"{'num_env_steps_sampled': 1000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 1000, 'num_agent_steps_trained': 0}",{},2023-05-08_17-48-18,False,nan,{},nan,nan,nan,0,0,seymour,"{'learner': {}, 'num_env_steps_sampled': 1000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 1000, 'num_agent_steps_trained': 0}",1,192.168.0.222,1000,0,1000,1000,0,0,0,1,0,0,0,"{'cpu_util_percent': 34.699999999999996, 'ram_util_percent': 39.93333333333333, 'gpu_util_percent0': 0.056666666666666664, 'vram_util_percent0': 0.0576171875}",679423,{},{},{},{},"{'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [], 'episode_lengths': []}, 'sampler_perf': {}, 'num_faulty_episodes': 0, 'connector_metrics': {}}",1.80517,1.80517,1.80517,"{'training_iteration_time_ms': 7.039, 'sample_time_ms': 6.532}",1683560898,1000,1,b9c9d_00000


(DQN pid=680153) 2023-05-08 17:48:31,835	WARNING multi_agent_prioritized_replay_buffer.py:215 -- Adding batches with column `weights` to this buffer while providing weights as a call argument to the add method results in the column being overwritten.
(DQN pid=746382) 2023-05-08 18:15:58,748	WARNING algorithm_config.py:635 -- Cannot create DQNConfig from given `config_dict`! Property __stdout_file__ not supported.
(DQN pid=746382) 2023-05-08 18:15:59,201	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=746574) 2023-05-08 18:16:03,733	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(DQN pid=746382) 2023-05-08 18:16:04,210	WARNING multi_agent_prioritized_replay_buffer.py:215 -- Adding batches with column `weights` to this buffer while providing weights as a call argument to

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'mean_q': 1.1499625444412231, 'min_q': 1.1472853422164917, 'max_q': 1.1981370449066162, 'cur_lr': 0.0005}, 'model': {}, 'num_grad_updates_lifetime': 57500.0, 'diff_num_grad_updates_vs_sampler_policy': 57499.0, 'td_error': array([ 5.2051014e+01,  2.9675703e+00,  3.1171062e+01,  5.6244116e+00,
       -4.1599060e+01, -2.1577101e+00, -3.3543293e+01,  1.6538687e+01,
        1.0576648e+00, -1.0789150e+01, -6.0554333e+00, -1.4377775e+01,
       -4.9149961e+00, -4.6388084e+01,  6.7986093e+00, -1.9250404e+00,
       -1.3620989e+01, -7.2477303e+00, -5.5489788e+00, -4.9443917e+00,
        1.1751221e+00,  8.6072655e+00, -4.7233269e+01, -7.3030949e-02,
       -3.7860980e+00, -1.8501219e+01, -3.5854874e+01, -6.2660813e-02,
       -3.2709913e+00, -1.3305000e+01, -4.4570055e+00,  9.6619129e-03],
      dtype=float32), 'mean_td_error': -6.05174160

In [6]:
env = StockExchangeEnv0()

env.reset(seed=52)
done = False
reward_total = 0
while not done:
    env.render()
    current_idx = env.current_idx
    current_price = env.price_data[current_idx]
    next_price = env.price_data[current_idx + 1]
    if next_price > current_price:
        action = 1
    elif next_price < current_price:
        action = 2
    else:
        action = 0
    state, reward, done, _, _ = env.step(action)
    reward_total += reward

print(f'{reward_total=:.2f}')

reward_total=4861.41


In [48]:
from rl_trading.simulation.env import SimulationConfig, ExchangeConfig
env = StockExchangeEnv0(sim_config=SimulationConfig(initial_cash=10_000), exchange_config=ExchangeConfig(maker_fee=1e-3))

env.reset(seed=52)
done = False
reward_total = 0
while not done:
    #env.render()
    current_idx = env.current_idx
    current_price = env.price_data[current_idx]
    next_price = env.price_data[current_idx + 1]
    balance = env.cash_balance
    position = env.asset_holdings
    rpc = (next_price - current_price) / current_price
    if next_price > current_price and balance > 0 and rpc > 1.2e-3:
        action = 1
    elif next_price < current_price and position > 0:
        action = 2
    else:
        action = 0
    state, reward, done, _, _ = env.step(action)
    reward_total += reward

print(f'{reward_total=:.2f}')

reward_total=625.41


In [77]:
from rl_trading.simulation.env import SimulationConfig, ExchangeConfig
env = StockExchangeEnv0(sim_config=SimulationConfig(initial_cash=10_000), exchange_config=ExchangeConfig(maker_fee=1e-3))

def find_next_buy_idx(cix):
    current_price = env.price_data[cix]
    buy_rpc, buy_idx = 0, cix
    for idx in range(cix, env.start_idx + env.max_steps - 1):
        next_price = env.price_data[idx + 1]
        if next_price > current_price:
            break
        rpc = (next_price - current_price) / current_price
        if rpc < buy_rpc:
            buy_rpc = rpc
            buy_idx = idx
    return buy_idx

def find_next_sell_idx(cix, fee=2e-3):
    current_price = env.price_data[cix]
    sell_rpc, sell_idx = 0, cix
    for idx in range(cix, env.start_idx + env.max_steps - 1):
        next_price = env.price_data[idx + 1]
        if next_price < current_price:
            break
        rpc = (next_price - current_price) / current_price
        if rpc > sell_rpc:
            sell_rpc = rpc
            sell_idx = idx
    return sell_idx if sell_rpc > fee else cix

env.reset(seed=52)
done = False
reward_total = 0
buy_idx, sell_idx = None, None
while not done:
    env.render()
    current_idx = env.current_idx
    current_price = env.price_data[current_idx]
    balance = env.cash_balance
    if balance > 0 and buy_idx is None and sell_idx is None:
        buy_idx = find_next_buy_idx(current_idx)
        sell_idx = find_next_sell_idx(buy_idx)
        if np.abs(buy_idx - sell_idx) < 5:
            buy_idx = sell_idx = None
        else:
            print(f'Buy after {buy_idx - env.current_idx} steps')
            print(f'Sell after {sell_idx - env.current_idx} steps')
            print(f'Trade period {sell_idx - buy_idx}')
            print()
    if buy_idx is not None and current_idx == buy_idx:
        buy_idx = None
        action = 1
    elif sell_idx is not None and current_idx == sell_idx:
        sell_idx = None
        action = 2
    else:
        action = 0
    state, reward, done, _, _ = env.step(action)
    reward_total += reward

print(f'{reward_total=:.2f}')

Buy after 0 steps
Sell after 58 steps
Trade period 58

Buy after 0 steps
Sell after 12 steps
Trade period 12

Buy after 0 steps
Sell after 35 steps
Trade period 35

Buy after 0 steps
Sell after 283 steps
Trade period 283

Buy after 0 steps
Sell after 9 steps
Trade period 9

Buy after 0 steps
Sell after 18 steps
Trade period 18

Buy after 0 steps
Sell after 21 steps
Trade period 21

Buy after 0 steps
Sell after 14 steps
Trade period 14

Buy after 0 steps
Sell after 18 steps
Trade period 18

Buy after 0 steps
Sell after 6 steps
Trade period 6

Buy after 0 steps
Sell after 20 steps
Trade period 20

Buy after 0 steps
Sell after 150 steps
Trade period 150

Buy after 0 steps
Sell after 15 steps
Trade period 15

Buy after 0 steps
Sell after 29 steps
Trade period 29

Buy after 0 steps
Sell after 26 steps
Trade period 26

Buy after 0 steps
Sell after 42 steps
Trade period 42

Buy after 0 steps
Sell after 11 steps
Trade period 11

Buy after 0 steps
Sell after 5 steps
Trade period 5

Buy after 0 

In [3]:
import numpy as np
from rl_trading.simulation.env import StockExchangeEnv1

env = StockExchangeEnv1(sim_config={'granularity': '1d', 'max_steps': 30})
done = False
env.reset()
while not done:
    env.render()
    state, reward, done, _, _ = env.step(np.random.randint(3))

In [2]:
from ray import tune, air
from ray.rllib.algorithms.dqn import DQNConfig
ray.shutdown()
ray.init()

from rl_trading.simulation.env import StockExchangeEnv1

config = (
    DQNConfig()
    .rollouts(num_rollout_workers=1, num_envs_per_worker=4)
    .resources(num_gpus=0.25)
    .environment(env='StockExchangeEnv-v1', env_config={'sim_config': {'granularity': '1d', 'max_steps': 30}})
)

tuner = tune.Tuner(
    "DQN",
    run_config=air.RunConfig(
        name='DQN_1h',
        local_dir='../exp_results/DQN',
        stop={"training_iteration": 250},
    ),
    tune_config=tune.TuneConfig(
        num_samples=5
    ),
    param_space=config,
)

tuner.fit()

2023-05-08 22:56:58,937	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
/home/fassty/anaconda3/envs/diploma_thesis/lib/python3.10/site-packages/ray/tune/experiment/experiment.py:170: UserWarning: The `local_dir` argument of `Experiment is deprecated. Use `storage_path` or set the `TUNE_RESULT_DIR` environment variable instead.
  warnings.warn(


(DQN pid=1442911) 2023-05-08 22:57:04,475	WARNING algorithm_config.py:635 -- Cannot create DQNConfig from given `config_dict`! Property __stdout_file__ not supported.
(DQN pid=1442911) 2023-05-08 22:57:04,940	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=1443069) 2023-05-08 22:57:10,315	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(DQN pid=1442911) 2023-05-08 22:57:10,849	WARNING multi_agent_prioritized_replay_buffer.py:215 -- Adding batches with column `weights` to this buffer while providing weights as a call argument to the add method results in the column being overwritten.
(DQN pid=1443313) 2023-05-08 22:57:13,644	WARNING algorithm_config.py:635 -- Cannot create DQNConfig from given `config_dict`! Property __stdout_file__ not supported.
(DQN pid=1443313) 2023-

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_total,training_iteration,trial_id
DQN_StockExchangeEnv-v1_e0354_00001,1000,"{'ObsPreprocessorConnector_ms': 0.004503875970840454, 'StateBufferConnector_ms': 0.007937848567962646, 'ViewRequirementAgentConnector_ms': 0.09392797946929932}","{'num_env_steps_sampled': 1000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 1000, 'num_agent_steps_trained': 0}",{},2023-05-08_22-57-21,False,30,{},3469.58,-430.682,-3457.52,32,32,seymour,"{'learner': {}, 'num_env_steps_sampled': 1000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 1000, 'num_agent_steps_trained': 0}",1,192.168.0.222,1000,0,1000,1000,0,0,0,1,0,0,0,"{'cpu_util_percent': 47.26666666666667, 'ram_util_percent': 43.0, 'gpu_util_percent0': 0.09333333333333334, 'vram_util_percent0': 0.08308919270833333}",1443313,{},{},{},"{'mean_raw_obs_processing_ms': 2.354254285652799, 'mean_inference_ms': 1.3133263682939142, 'mean_action_processing_ms': 0.2668654301252023, 'mean_env_wait_ms': 0.4541959420618308, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 3469.5800320231665, 'episode_reward_min': -3457.5212727048793, 'episode_reward_mean': -430.68205518945325, 'episode_len_mean': 30.0, 'episode_media': {}, 'episodes_this_iter': 32, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-2500.04274758891, -512.8307417985961, -1273.6865750049637, 1733.4447958525543, 10.114891306227946, 2589.2849614948427, -70.10214516004271, -795.4055995302479, -410.4967926872705, -995.569969954844, -246.43124687011368, -1652.4171011887302, -342.1916201364711, -2311.150006707241, 180.80874994796613, -187.2972336815892, 283.6635246990827, -731.6369512781384, 973.9020299930344, 1074.2199611716442, -265.49101096542836, -3457.5212727048793, -371.5881167043117, -1460.6249755388435, -955.2961529465683, -1147.1377868969466, 691.4383685029188, -814.0020311571025, 3469.5800320231665, -3324.026625412879, 19.458779973305354, -982.7951571131289], 'episode_lengths': [30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 2.354254285652799, 'mean_inference_ms': 1.3133263682939142, 'mean_action_processing_ms': 0.2668654301252023, 'mean_env_wait_ms': 0.4541959420618308, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.004503875970840454, 'StateBufferConnector_ms': 0.007937848567962646, 'ViewRequirementAgentConnector_ms': 0.09392797946929932}}",1.8581,1.8581,1.8581,"{'training_iteration_time_ms': 7.872, 'sample_time_ms': 7.343}",1683579441,1000,1,e0354_00001


(DQN pid=1443734) 2023-05-08 22:57:27,669	WARNING multi_agent_prioritized_replay_buffer.py:215 -- Adding batches with column `weights` to this buffer while providing weights as a call argument to the add method results in the column being overwritten.
(DQN pid=1513872) 2023-05-08 23:26:24,034	WARNING algorithm_config.py:635 -- Cannot create DQNConfig from given `config_dict`! Property __stdout_file__ not supported.
(DQN pid=1513897) 2023-05-08 23:26:24,533	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=1514084) 2023-05-08 23:26:29,236	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(DQN pid=1513897) 2023-05-08 23:26:29,236	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will no

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'mean_q': 997.37890625, 'min_q': 645.4097290039062, 'max_q': 4936.7841796875, 'cur_lr': 0.0005}, 'model': {}, 'num_grad_updates_lifetime': 62250.0, 'diff_num_grad_updates_vs_sampler_policy': 62249.0, 'td_error': array([ -840.6677   ,  -864.65283  ,  -617.3421   ,  -398.42664  ,
         422.38892  ,   294.2863   , -1647.2065   ,     7.1083374,
         314.30743  ,   557.4609   ,  -215.11768  ,   -40.32794  ,
         427.96448  ,  -343.78387  ,   156.38074  ,  -381.0603   ,
           7.343628 , -1283.1091   ,   163.56226  ,   223.42618  ,
         -34.72101  ,   291.45343  ,  -568.25867  , -2439.538    ,
         260.00385  ,   -98.30902  ,  -222.34882  ,   584.7792   ,
         133.24792  , -3316.2568   ,  -291.16803  ,  -100.52789  ],
      dtype=float32), 'mean_td_error': -308.09716796875}}, 'num_env_steps_sampled': 250000, 

In [10]:
import torch
import torch.nn as nn

class Network:
    def __init__(self) -> None:
        # Use GPU if available.
        self._device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self._model = nn.Sequential(
            nn.Linear(5, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 3)
        ).to(self._device)

        self._optimizer = torch.optim.Adam(self._model.parameters(), lr=1e-3)
        self._loss = nn.MSELoss()

    def train(self, states: np.ndarray, actions: np.ndarray, q_values: np.ndarray) -> None:
        states = torch.from_numpy(states).float().to(self._device)
        q_values = torch.from_numpy(q_values).float().to(self._device)
        actions = torch.from_numpy(actions).long().to(self._device)

        self._model.train()
        self._optimizer.zero_grad()
        predictions = self._model(states)
        predictions = torch.gather(predictions, dim=1, index=actions)
        loss = self._loss(predictions, q_values)
        loss.backward()
        #nn.utils.clip_grad_norm_(self._model.parameters(), 10)
        self._optimizer.step()

    def predict(self, states: np.ndarray) -> np.ndarray:
        states = torch.from_numpy(states).float().to(self._device)
        self._model.eval()
        with torch.no_grad():
            return self._model(states).cpu().numpy()

    def copy_weights_from(self, other) -> None:
        params = dict(self._model.named_parameters())
        params_other = dict(other._model.named_parameters())
        with torch.no_grad():
            for name, value in params_other.items():
                params[name].data.copy_(value.data)

In [11]:
import collections
import random

network = Network()

epsilon = 0.5
gamma = 0.99

replay_buffer = collections.deque()

Transition = collections.namedtuple("Transition", ["state", "action", "reward", "done", "next_state"])

for episode in range(20):
    state, done = env.reset()
    episode_return = 0

    while not done:
        with torch.no_grad():
            q_values = network.predict(np.array([state], dtype=np.float32))[0]
        if np.random.uniform() >= epsilon:
            action = np.argmax(q_values)
        else:
            action = np.random.randint(0, 3)

        next_state, reward, done = env.step(action)
        episode_return += reward
        replay_buffer.append(Transition(state, action, reward, done, next_state))


        if len(replay_buffer) > 512:
            minibatch = random.sample(replay_buffer, 512)
            states = np.vstack([t.state for t in minibatch])
            actions = np.vstack([t.action for t in minibatch])
            rewards = np.vstack([t.reward for t in minibatch])
            next_states = np.vstack([t.next_state for t in minibatch])
            dones = np.vstack([t.done for t in minibatch]).astype(np.uint8)

            with torch.no_grad():
                q_values_next = network.predict(next_states)
                q_values_next = q_values_next.max(axis=1).reshape(-1, 1)
                target_q_values = rewards + (1 - dones) * gamma * q_values_next
            network.train(states, actions, target_q_values)

        state = next_state
    print(f'Episode: {episode} return: {episode_return}')

Episode: 0 return: 224.99744838047303
Episode: 1 return: 264.531597508022
Episode: 2 return: 10.526600750194191
Episode: 3 return: 463.344433536383
Episode: 4 return: 107.36902379122189
Episode: 5 return: 659.0909868725554
Episode: 6 return: 93.25250433980881
Episode: 7 return: -754.3228877029144
Episode: 8 return: -12.09655519360639
Episode: 9 return: -4.448079854905141
Episode: 10 return: 404.5836446362279
Episode: 11 return: 202.75678159449467
Episode: 12 return: -403.27718192251444
Episode: 13 return: -319.655870002426
Episode: 14 return: -126.48002786972344
Episode: 15 return: -267.77574004643776
Episode: 16 return: 175.35113259349407
Episode: 17 return: 712.920821829259
Episode: 18 return: 370.2768818837727
Episode: 19 return: -240.6320419818552


In [12]:
4261.48 - 4280.56

-19.080000000000837

In [34]:
env.reset()

(array([ 3829.47615283,   872.362006  ,  3829.47615283,  3829.47615283,
            0.        ,    50.        ,  3829.47615283, 10000.        ,
            0.        ]),
 {})

In [89]:
env.step(1)

(array([ 3.80494806e+03,  3.85813469e+02,  3.80876260e+03,  3.81634181e+03,
        -6.61621262e-01,  4.21757695e+01,  3.81111808e+03,  0.00000000e+00,
         2.62047935e+00]),
 -13.047519166573693,
 False,
 False,
 {})

In [1]:
import pandas as pd

prices_df = pd.read_hdf('data/binance_BTC_USDT.h5')


,amount,price
timestamp,,
1502942460,1.775183,4261.480000
1502942580,0.261074,4280.560000
1502942640,0.012008,4261.480000
1502942700,0.140796,4261.480000
1502943480,0.075455,4262.187216
...,...,...
1670479020,140.372990,16822.509019
1670479080,135.652550,16823.593779
1670479140,106.761210,16823.667239


In [22]:
prices_df.index = pd.to_datetime(prices_df.index * 1e9)
prices_df

,amount,price
timestamp,,
2017-08-17 04:01:00,1.775183,4261.480000
2017-08-17 04:03:00,0.261074,4280.560000
2017-08-17 04:04:00,0.012008,4261.480000
2017-08-17 04:05:00,0.140796,4261.480000
2017-08-17 04:18:00,0.075455,4262.187216
...,...,...
2022-12-08 05:57:00,140.372990,16822.509019
2022-12-08 05:58:00,135.652550,16823.593779
2022-12-08 05:59:00,106.761210,16823.667239


In [28]:
hourly_prices = prices_df.groupby(pd.Grouper(freq='H')).agg({'amount': 'sum', 'price': 'last'})

In [99]:
hourly_prices['price'] = hourly_prices['price'].ffill()

In [91]:

            #time.sleep(0.01)

env = StockExchangeEnv(hourly_prices['price'].to_numpy(), hourly_prices['amount'].to_numpy(), 10_000)

env.reset()
for i in range(100):
    env.render()
    current_step = env.current_step
    current_price = env.price_data[current_step]
    next_price = env.price_data[current_step + 1]
    if next_price > current_price:
        env.step(1)
    elif next_price < current_price:
        env.step(2)
    else:
        env.step(0)

# for i in range(100):
#     env.render()
#     current_step = env.current_step
#     current_price = env.price_data[current_step]
#     next_price = env.price_data[current_step + 1]
#     try:
#         next_next_price = env.price_data[current_step + 2]
#     except IndexError:
#         next_next_price = 0
#     print(f'{current_price=:.2f}\t{next_price=:.2f}\t{next_next_price=:.2f}')
#     print('Enter next action: 0 HOLD, 1 BUY, 2 SELL\n')
#     time.sleep(0.1)
#     action = int(input())
#     env.step(action)
plt.show()

Using matplotlib backend: TkAgg


In [38]:
from rl_trading.

2023-05-06 16:36:43,057	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


NameError: name 'hourly_prices' is not defined

2023-05-05 23:20:54,623	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


2023-05-05 23:20:56,043	WARNING util.py:67 -- Install gputil for GPU system monitoring.


Step: 1	Mean return: -42.865930799546184
Step: 2	Mean return: -185.89766800128746
Step: 3	Mean return: 169.17498209446853
Step: 4	Mean return: -88.3142230583735
Step: 5	Mean return: 19.19788326543157
Step: 6	Mean return: -16.45068610735584
Step: 7	Mean return: 7.063555817371994
Step: 8	Mean return: -38.02530962088903
Step: 9	Mean return: -41.522897312310235
Step: 10	Mean return: -17.97407142151989
Step: 11	Mean return: -19.678367718046115
Step: 12	Mean return: 11.896916841168824
Step: 13	Mean return: -2.235516732331765
Step: 14	Mean return: 12.66614309286765
Step: 15	Mean return: -4.661710404750021
Step: 16	Mean return: -16.956172854057762
Step: 17	Mean return: -11.008859576698496
Step: 18	Mean return: 2.480511230930242
Step: 19	Mean return: -0.4883398345264868
Step: 20	Mean return: 26.459753117181773
Step: 21	Mean return: -44.16181603506087
Step: 22	Mean return: 20.183706555358384
Step: 23	Mean return: -3.228635785139504
Step: 24	Mean return: 24.918146317321316
Step: 25	Mean return: 8

Exception ignored in: <function Variable.__del__ at 0x7f17eafc2950>
Traceback (most recent call last):
  File "/home/fassty/anaconda3/envs/diploma_thesis/lib/python3.10/tkinter/__init__.py", line 388, in __del__
    if self._tk.getboolean(self._tk.call("info", "exists", self._name)):
RuntimeError: main thread is not in main loop


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 11.0 MB/s eta 0:00:0000:0100:01
  Using cached gym-0.26.2.tar.gz (721 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 5.9 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 12.2 MB/s eta 0:00:0000:0100:01�╸ 61.8/61.8 MB 115.6 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 8.5 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.0/297.0 kB 6.3 MB/s eta 0:00:0000:01
  Using

In [100]:
hourly_prices['price'].isna().any()

False